<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Multilabel-Design-Pattern" data-toc-modified-id="Multilabel-Design-Pattern-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Multilabel Design Pattern</a></span></li><li><span><a href="#問題描述" data-toc-modified-id="問題描述-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>問題描述</a></span></li><li><span><a href="#解決方案---NN" data-toc-modified-id="解決方案---NN-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>解決方案 - NN</a></span></li><li><span><a href="#代價、注意事項與其他方案" data-toc-modified-id="代價、注意事項與其他方案-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>代價、注意事項與其他方案</a></span><ul class="toc-item"><li><span><a href="#注意事項-:-sigmoid-function-和二元分類的不同之處" data-toc-modified-id="注意事項-:-sigmoid-function-和二元分類的不同之處-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>注意事項 : sigmoid function 和二元分類的不同之處</a></span></li><li><span><a href="#注意事項-:-損失函數" data-toc-modified-id="注意事項-:-損失函數-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>注意事項 : 損失函數</a></span></li><li><span><a href="#注意事項-:-資料平衡度" data-toc-modified-id="注意事項-:-資料平衡度-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>注意事項 : 資料平衡度</a></span></li><li><span><a href="#層狀標籤" data-toc-modified-id="層狀標籤-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>層狀標籤</a></span></li><li><span><a href="#其他方案-:-One-Versus-Rest" data-toc-modified-id="其他方案-:-One-Versus-Rest-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>其他方案 : One Versus Rest</a></span></li></ul></li><li><span><a href="#其他解決方案---One-Versus-Rest" data-toc-modified-id="其他解決方案---One-Versus-Rest-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>其他解決方案 - One Versus Rest</a></span></li></ul></div>

In [1]:
# auto-compeletion faster
%config Completer.use_jedi = False

In [14]:
import os
from os.path import join as PJ
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Embedding, Input, Flatten, Conv2D, MaxPooling2D

from sklearn.utils import shuffle
from sklearn.preprocessing import MultiLabelBinarizer


# Multilabel Design Pattern


* 解決一筆資料可能有多個標籤問題

* 以下會介紹 NN 以及 一般分類器的解決模式

# 問題描述 


當我們使用 softmax 作為輸出層的 Actvation function 時，模型的輸出是一個`n`個元素的陣列，加總為1，每一個值都代表特定訓練實體屬於的類別機率(先以NN來看的話是這個樣子的)
    
舉例 : 

對於一個模型可將圖性輸出分類為貓、狗、兔子，那麼對於特定圖像 : softmax的輸出可能是 : `[.89, .02, .09]`

那麼在這個場景下，**每張圖像只有一個可能的標籤**，我們可以用argmax來決定預測類別

----------------------------------------------------------------------

轉換到另一個場景 : 

1. 以BigQuery的Stackoverflow問題資料組為例，一個訓練實體例如`How do I plot a pandas DataFrame`，可能會被附加`Python`,`pandas`, `visulization`標籤 (本案例)

2. 識別惡意評論的模型，我們可能會想使用多種惡意性質的標籤來標記評論，例如`仇恨`、`猥褻`等

3. 識別患者可能有的潛在疾病，例如醫療資料組(每位患者的身高、體重、年齡、血壓等等)，用來預測一位患者同時有`心臟病`、`糖尿病`等風險的模型

4. [從影片的meta data中識別影片類型，例如Doc Strange 屬於 `動作`, `冒險`, `奇幻` 等類型](https://towardsdatascience.com/journey-to-the-center-of-multi-label-classification-384c40229bff)

總之 : 當你的資料 - 單一個訓練實體可以屬於多個類別(標註者這麼認為)，而且每一種解釋都正確時，那麼你可以考慮使用Multi-label的解決方案

# 解決方案 - NN

解決方案其實異常的容易，單一標籤的分類問屜最後輸出使用`softmax`，我們可以改用`sigmoid`，如此一來每一個輸出類別的值域就會在 0~1 之間，以下我們舉例

# 代價、注意事項與其他方案

## 注意事項 : sigmoid function 和二元分類的不同之處

* 二元分類我們也使用 `sigmoid`, 但輸出層的寫法會是 

`keras.layers.Dense(1, activation ='sigmoid')`

* 然而Multilabel時的最後則會是

`keras.layer.Desnse(YN, activation='sigmoid')`, $YN$ : 你的標籤數量

## 注意事項 : 損失函數

* 單一標籤多類別分類時會使用`catogorical_crossentropy`，而多標籤時，則是使用`binary_crossentroy`，祝是因為，一個有三個類別的多標籤問題，實質上是屬於三個小的二元分類問題

<img src='./assets/006_1.jpg'></img>

##  注意事項 : 資料平衡度

以 Stackoverflow 資料組為例，他可能有許多同時被標記為Tensorflow 和 Keras 問題，但也有一些 Keras 問題和 Tensorflow 無關，也就是說如果在你的資料集中，大部分Keras問題都有被貼上Tensorflow的標籤，那麼你的模型可能無法正確地判斷一個資料實體只屬於Keras的情況

## 層狀標籤

有些問題看似多標籤，但事實上該標籤具有階層的特性，比如從部落格文章中做食物圖片辨識

* 圖片
    * 食物
        * 麵
        * 飯
            * 咖喱飯
            * 滷肉飯
            * ...
        * 炸物
            * 炸雞
            * ...
    * 環境
    * 飲料
    * 含人類的合照

如果你的標籤是屬於層狀的，那麼串連模式(Cascade)或許更適合解決你的問題


## 其他方案 : One Versus Rest

在我們當前的架構下，我們的標籤是可以互相影響的，比如一個屬於tensorflow標籤的訓練實體很有可能也和Keras有關

另一種作法則是採用 One Versus Rest - 將每個標籤都視為一個問題，用一個模型去解他，有$YN$個標籤我們就有$YN$個模型
    * 好處 : 這麼做可以協助處理罕見類別的問題，越罕見的類別可以設置越大的`classs_weight`去調整，或者針對該類別做`Reblancing`
    * 壞處 (可能也是好處，這和你的任務有關) - 每個預測標籤不在彼此有關係，內文是屬於tensorflow還是keras彼此不具有關係



In [7]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = ".key/credential.json"
# from google.cloud import storage
# storage_client = storage.Client()

In [12]:
# DO it in your terminal
# DOWNLOAD = True
# !gsutil cp 'gs://ml-design-patterns/so_data.csv' .

In [52]:
DATA = PJ("data",'006_multilabel.csv')
SEED = 42

In [53]:
# df = bq.query(query).to_dataframe()
# df.to_csv(DATA, index=False)

df = pd.read_csv(DATA)
print(df.shape)
df.head()

(188199, 3)


,extracted_tags,original_tags,text
0,"matplotlib,pandas","python,matplotlib,pandas",setting xticks and yticks for scatter plot mat...
1,"scikitlearn,keras","python,numpy,scikit-learn,keras,grid-search",gridseachcv - valueerror: found input variable...
2,"matplotlib,scikitlearn","python,numpy,matplotlib,scikit-learn,nmf",non negative matrix factorisation in python on...
3,"pandas,tensorflow","python,pandas,tensorflow,time-series",avocado equivalent to avocado.dataframe.resamp...
4,"matplotlib,pandas","python,matplotlib,plot,pandas",how to plot on avocado python i have a data fr...


In [54]:
data = (
    shuffle(
    df
    .drop(columns='original_tags')
    .rename(columns={'extracted_tags' : 'tags'})
    .dropna(),
    
    random_state = SEED
    )
)

data.head()

,tags,text
25318,pandas,how to calculate sum of a rows according to th...
1819,keras,"avocado neural nets, how to remove nan values ..."
15345,pandas,extracting a ration in two column from a text ...
185432,"pandas,matplotlib",graphs from two sets of data files with differ...
180414,"tensorflow,keras",separable depthwise convolutions without mixin...


In [55]:
tags_split = [tags.split(',') for tags in data['tags'].values]
print(tags_split[0])

['pandas']


In [56]:
tag_encoder = MultiLabelBinarizer()
tags_encoded = tag_encoder.fit_transform(tags_split)
num_tags = len(tags_encoded[0])
print(data['text'].values[0][:110])
print(tag_encoder.classes_)
print(tags_encoded[0])

how to calculate sum of a rows according to their index in a dataframe python i want to know how to calculate 
['keras' 'matplotlib' 'pandas' 'scikitlearn' 'tensorflow']
[0 0 1 0 0]


In [57]:
# Split our data into train and test sets
train_size = int(len(data) * .8)
print ("Train size: %d" % train_size)
print ("Test size: %d" % (len(data) - train_size))

Train size: 150559
Test size: 37640


In [58]:
# Split our labels into train and test sets
train_tags = tags_encoded[:train_size]
test_tags = tags_encoded[train_size:]


In [59]:
train_qs = data['text'].values[:train_size]
test_qs = data['text'].values[train_size:]


In [60]:
from tensorflow.keras.preprocessing import text

VOCAB_SIZE=400 # This is a hyperparameter, try out different values for your dataset

tokenizer = text.Tokenizer(num_words=VOCAB_SIZE)
tokenizer.fit_on_texts(train_qs)

body_train = tokenizer.texts_to_matrix(train_qs)
body_test = tokenizer.texts_to_matrix(test_qs)


In [115]:
print(dir(tokenizer))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_keras_api_names', '_keras_api_names_v1', 'char_level', 'document_count', 'filters', 'fit_on_sequences', 'fit_on_texts', 'get_config', 'index_docs', 'index_word', 'lower', 'num_words', 'oov_token', 'sequences_to_matrix', 'sequences_to_texts', 'sequences_to_texts_generator', 'split', 'texts_to_matrix', 'texts_to_sequences', 'texts_to_sequences_generator', 'to_json', 'word_counts', 'word_docs', 'word_index']


In [61]:
# For this case 
# We just use 5 class for multilabel classification
# tags : (N, Y) -> num of instance, num of labels
# body_train : (N, D) -> num of instance, num of features
for arr in [train_tags, train_qs, body_train]:
    print(type(arr))
    print(arr.shape)
    if arr.ndim > 1:
        print(arr[0, :])

<class 'numpy.ndarray'>
(150559, 5)
[0 0 1 0 0]
<class 'numpy.ndarray'>
(150559,)
<class 'numpy.ndarray'>
(150559, 400)
[0. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 

In [62]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(50, input_shape=(VOCAB_SIZE,), activation='relu'))
model.add(tf.keras.layers.Dense(25, activation='relu'))
model.add(tf.keras.layers.Dense(num_tags, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [63]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 50)                20050     
_________________________________________________________________
dense_7 (Dense)              (None, 25)                1275      
_________________________________________________________________
dense_8 (Dense)              (None, 5)                 130       
Total params: 21,455
Trainable params: 21,455
Non-trainable params: 0
_________________________________________________________________


In [64]:
model.fit(body_train, train_tags, epochs=10, batch_size=128, validation_split=0.1)
print('Eval loss/accuracy:{}'.format(
  model.evaluate(body_test, test_tags, batch_size=128)))

Train on 135503 samples, validate on 15056 samples
Epoch 1/10
135503/135503 [==============================] - 2s 16us/sample - loss: 0.1544 - accuracy: 0.9408 - val_loss: 0.1115 - val_accuracy: 0.9585
Epoch 2/10
135503/135503 [==============================] - 2s 15us/sample - loss: 0.1062 - accuracy: 0.9592 - val_loss: 0.1052 - val_accuracy: 0.9591
Epoch 3/10
135503/135503 [==============================] - 2s 14us/sample - loss: 0.1001 - accuracy: 0.9608 - val_loss: 0.1023 - val_accuracy: 0.9593
Epoch 4/10
135503/135503 [==============================] - 2s 14us/sample - loss: 0.0964 - accuracy: 0.9618 - val_loss: 0.0991 - val_accuracy: 0.9610
Epoch 5/10
135503/135503 [==============================] - 2s 15us/sample - loss: 0.0933 - accuracy: 0.9631 - val_loss: 0.0987 - val_accuracy: 0.9607:
Epoch 6/10
135503/135503 [==============================] - 2s 15us/sample - loss: 0.0908 - accuracy: 0.9639 - val_loss: 0.0977 - val_accuracy: 0.9610
Epoch 7/10
135503/135503 [================

In [73]:
# Get some test predictions
predictions = model.predict(body_test[0:1000])

classes = tag_encoder.classes_

In [81]:
threshold = 0.7
n_shown = 0
for q_idx, probabilities in enumerate(predictions):
    n_pred = probabilities[probabilities > threshold]
    if len(n_pred) > 1:
        print(f'------------------ instance {q_idx}-----------------')
        print()
        print(test_qs[q_idx])
        print()
        print(f'raw prediction {probabilities} ')
        for idx, tag_prob in enumerate(probabilities):
            if tag_prob > threshold:
                print(classes[idx], round(tag_prob * 100, 2), '%')
        n_shown += 1
        if n_shown > 3:
            break

------------------ instance 102-----------------

applying function on avocado column using information from another column i have a dataframe that contains a bunch of people's text descriptions. other than that, i also have 4 descriptions a,b,c,d.  for each person's text description, i wish to compare them to each of the 4 descriptions by using cosine similarity and store these scores in the same dataframe in 4 new columns: a, b, c, d.   how can i do this in a panda way, without using for loops? i was thinking of using the apply function but i don't know how to reference to the 'text' column as well as the 4 descriptions a,b,c,d in the apply function.  thank you very much for any help!!  what i have tried:   import avocado as avocado from avocado.feature_extraction.text import countvectorizer from avocado.metrics.pairwise import cosine_similarity  person_one = [' '.join(['table','car','mouse'])] person_two = [' '.join(['computer','card','can','mouse'])] person_three = [' '.join(['chai

# 其他解決方案 - One Versus Rest

Ref : 

1. [Deep Dive into MuliLabel Classification](https://towardsdatascience.com/journey-to-the-center-of-multi-label-classification-384c40229bff)

2. [Multi Label Text Classification with Scikit-Learn](https://towardsdatascience.com/multi-label-text-classification-with-scikit-learn-30714b7819c5)


In [96]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [97]:
classes
for cat_idx, category in enumerate(classes):
    print(cat_idx, category)
    print(train_tags[:,cat_idx])

0 keras
[0 1 0 ... 0 0 0]
1 matplotlib
[0 0 0 ... 0 1 1]
2 pandas
[1 0 1 ... 1 0 0]
3 scikitlearn
[0 0 0 ... 0 0 0]
4 tensorflow
[0 0 0 ... 0 0 0]


In [98]:
for arr in [train_tags, train_qs, body_train, body_test, test_tags]:
    print(type(arr))
    print(arr.shape)
    if arr.ndim > 1:
        print(arr[0, :])

<class 'numpy.ndarray'>
(150559, 5)
[0 0 1 0 0]
<class 'numpy.ndarray'>
(150559,)
<class 'numpy.ndarray'>
(150559, 400)
[0. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 

In [107]:
# Define a pipeline combining a text feature extractor with multi lable classifier
NB_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
            ])

# Since It's too slow, so not a option here
# even slower than the TensorflowDNN with few Dense layer
# LogReg_pipeline = Pipeline([
#                 ('clf', OneVsRestClassifier(LogisticRegression(solver='sag')
#                                             , n_jobs=-1)),
#             ])

models = {}
for model_name,model in zip(
    ['NB'],
    [NB_pipeline]):
    for cat_idx, category in enumerate(classes):
        print('... Processing {}'.format(category))
        # train the model using X_dtm & y
        model.fit(body_train, train_tags[:,cat_idx])
        # compute the testing accuracy
        prediction = model.predict(body_test)
        print(f'{model_name}')
        print('    Test accuracy is {}'.format(accuracy_score(test_tags[:, cat_idx], prediction)))
        models[category] = model


... Processing keras
NB
    Test accuracy is 0.8836875664187035
... Processing matplotlib
NB
    Test accuracy is 0.9543836344314559
... Processing pandas
NB
    Test accuracy is 0.9257970244420829
... Processing scikitlearn
NB
    Test accuracy is 0.9389213602550478
... Processing tensorflow
NB
    Test accuracy is 0.8893464399574921


In [114]:
# # Prediction
# # TODO